# Data Science Capstone Project


# Introduction / Business Problem


After I learned, practice and explored the dataset from this course. Now that makes me to interest this city (Toronto). So I would like to explore more than a recent assignment. And my purpose is " Where the place to setup/build a new office and co-working space in Toronto". This topic was interesting. Assume we are start-up or inverter. We need to know a place and location that are very comfortable for us, like nearby office should have a place such as Shopping Mall, Gym, Restaurateurs, Canteen.

In this project, we will try to build a feature for recommendation engine. By the Cluster and Segmention.

# Data

Now we have understood the purpose. It is time to analyze data for it. We will be using List_of_postal_codes_of_Canada from Wikipedia. It same recent assignment. I will use an old data, But try to explore new borough (not Downtown). to practice more analytic skill. And the method is understanding the data and preparing it for Clustering model. The pre-process step I will use the same method from the last assignment. But normalization step I will try to make effective fold of data. To explore a data relation the modelling. How many fold, it useful for this model or method. Ensure I will use location to visualize from API Foursquare.

## Let's Go! 

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans 
import seaborn as sns 
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 

### Implement Web-Scraping ##


In [2]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #URL
soup=BeautifulSoup(url,'lxml') 
table = soup.find("table",{"class":"wikitable sortable"}) #table Class
text_data = [] #declare List for storage text from table  
text_sys = table.findAll('tr') #detect all <tr> syntax include text

for text_loop in text_sys:
    ex_text = text_loop.findChildren(recursive=False) #explode list from text_sys 
    text_store = [] #declare list for store string text as list
    for read in ex_text:   #2nd loop in ex_text
        text_index = read.text.strip() #arrange text and strip
        text_store.append(text_index) #combine 3-text string as list  
            
    text_data.append(text_store) #out of 2nd loop save whole list text to one variable list 
text_data #try print out data

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 [

### Implement Data-to-DataFrame

In [3]:
df=pd.DataFrame(text_data,columns=["Postcode","Borough","Neighbourhood"]) #Assign data to dataframe
df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [4]:
#If Neighbour index have "Not assigned" it will instead a index of Borough.
df.loc[df.Neighbourhood=="Not assigned","Neighbourhood"] = df.Borough 
df.drop(df[df.Borough == 'Borough'].index, inplace=True) #remove name column in index
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True) #remove Borough rows if index =="Not assigned"
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [5]:
#sort dataframe 
df_ = df.sort_values(by=['Postcode'])
df_.reset_index(inplace=True)
df_.drop('index',axis=1,inplace=True)
df_.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek


In [6]:
df_.shape

(212, 3)

In [7]:
#sort dataframe by postcode
df = (df_.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(set(x.dropna()))).reset_index())
df.columns=["Postcode","Borough","Neighbourhood"]
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"Guildwood,West Hill,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Build Location for mapping ###

In [9]:
#load geo_data
geo_df = pd.read_csv("http://cocl.us/Geospatial_data")
geo_df.columns=["Postcode","Latitude","Longitude"]
geo_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#merge_data
df=pd.merge(df,geo_df,on="Postcode")
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,West Hill,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
#arrage to mapping
df_geo=df[["Borough","Neighbourhood","Latitude","Longitude"]]
df_geo.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
2,Scarborough,"Guildwood,West Hill,Morningside",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent='eiei')
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(Latitude, Longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


## Map Toronto

In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(df_geo, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [14]:
df['Borough'].value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

#### North York Toronto. Is my choice###

In [18]:
df_north=df[df['Borough']=='North York'].reset_index(drop=True)
df_north.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Oriole,Henry Farm,Fairview",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills,Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale,Newtonbrook",43.789053,-79.408493


In [21]:
address = 'North York, ON'
geolocator = Nominatim(user_agent='eiei')
location = geolocator.geocode(address)
Latitude = location.latitude
Longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(Latitude, Longitude))
# create map of Toronto using latitude and longitude values
map_north_Toronto = folium.Map(location=[Latitude, Longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_north['Latitude'], df_north['Longitude'], df_north['Borough'], df_north['Neighbourhood']):
    label = '{}, {}'.format(df_north, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_north_Toronto)  
    
map_north_Toronto

The geograpical coordinates of Toronto are 43.7709163, -79.4124102.


### Foursquare API to explore Downtown Toronto ###

In [22]:
CLIENT_ID = 'PQHF3XY3005LN3ZFE5RXCAEEYUKGHM3RZW2NHYTRWV1U325A' #Foursquare ID
CLIENT_SECRET = 'OSTVDXLO41ZEQER32DK3QCRYW1LF3D4IOO4OTBYICHLQYUUD' # Foursquare Secret
VERSION = '20180602' 

In [23]:
df_north['Neighbourhood'][0]

'Hillcrest Village'

In [24]:
neighborhood_latitude = df_north['Latitude'][0] 
neighborhood_longitude = df_north['Longitude'][0] 

In [25]:
LIMIT = 100   
radius = 5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PQHF3XY3005LN3ZFE5RXCAEEYUKGHM3RZW2NHYTRWV1U325A&client_secret=OSTVDXLO41ZEQER32DK3QCRYW1LF3D4IOO4OTBYICHLQYUUD&v=20180602&ll=43.8037622,-79.3634517&radius=5000&limit=100'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bcda5a34c1f67197d9e73ec'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 234,
  'suggestedBounds': {'ne': {'lat': 43.848762245000046,
    'lng': -79.30121655378305},
   'sw': {'lat': 43.75876215499996, 'lng': -79.42568684621695}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '551eda39498e4a752413a0ef',
       'name': 'Chatime Willowdale',
       'location': {'address': '175 Ravel Road',
        'crossStreet': 'Leslie and Finch',
        'lat': 43.791325874580686,
        'lng': -79.36750616764424,
        'labeledLatLngs': [{'label': 'displ

**get_category_type from Foursqure API** 

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**Cleansing JSON** 

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Chatime Willowdale,Bubble Tea Shop,43.791326,-79.367506
1,Bayview Golf & Country Club,Golf Course,43.809391,-79.375285
2,Gyubee 牛兵衞燒肉工房,BBQ Joint,43.815661,-79.349423
3,J-Town,Shopping Plaza,43.815542,-79.344459
4,Tastee,Bakery,43.807722,-79.356798


**And how many venues were returned by Foursquare?**

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [30]:
nearby_venues.shape

(100, 4)

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
north_venues = getNearbyVenues(names=df_north['Neighbourhood'],
                                   latitudes=df_north['Latitude'],
                                   longitudes=df_north['Longitude']
                                  )

Hillcrest Village
Oriole,Henry Farm,Fairview
Bayview Village
York Mills,Silver Hills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
York University,Northwood Park
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park,Lawrence Manor East
Lawrence Manor,Lawrence Heights
Glencairn
Upwood Park,Maple Leaf Park,North Park
Humber Summit
Humberlea,Emery


#### Let's check the size of the resulting dataframe

In [36]:
print(north_venues.shape)
north_venues.head()

(240, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports


In [37]:
print('There are {} uniques categories.'.format(len(north_venues['Venue Category'].unique())))

There are 111 uniques categories.


## Analyze Each Neighborhood

In [40]:
# one hot encoding
north_onehot = pd.get_dummies(north_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_onehot['Neighborhood'] = north_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_onehot.columns[-1]] + list(north_onehot.columns[:-1])
north_onehot = north_onehot[fixed_columns]

print(north_onehot.shape)
north_onehot.head()

(240, 112)


,Neighborhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
north_grouped = north_onehot.groupby('Neighborhood').mean().reset_index()
print(north_grouped.shape)
north_grouped.head()

(24, 112)


,Neighborhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.052632,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.052632,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.250000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.038462,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.038462,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CFB Toronto,Downsview East",0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Don Mills North,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in north_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_grouped[north_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                venue  freq
0         Coffee Shop  0.11
1  Frozen Yogurt Shop  0.05
2            Pharmacy  0.05
3         Pizza Place  0.05
4                Park  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1           Coffee Shop  0.08
2    Italian Restaurant  0.08
3              Pharmacy  0.04
4                  Café  0.04


----CFB Toronto,Downsview East----
      venue  freq
0   Airport  0.33
1      Park  0.33
2  Bus Stop  0.33
3     Plaza  0.00
4  Pharmacy  0.00


----Don Mills North----
                  venue  freq
0                  Pool  0.17
1  Gym / Fitness Center  0.17
2  Caribbean Restaurant  0.17
3                  Café  0.17
4   Japanese Restaurant  0.17


--

#### Let's put that into a *pandas* dataframe

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_grouped['Neighborhood']

for ind in np.arange(north_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Pet Store,Shopping Mall,Diner,Ice Cream Shop,Deli / Bodega,Park,Frozen Yogurt Shop,Pharmacy,Pizza Place
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Furniture / Home Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Gift Shop,Cosmetics Shop
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Sandwich Place,Liquor Store,Comfort Food Restaurant,Café,Butcher,Pharmacy,Pizza Place
3,"CFB Toronto,Downsview East",Airport,Park,Bus Stop,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
4,Don Mills North,Caribbean Restaurant,Basketball Court,Pool,Café,Japanese Restaurant,Gym / Fitness Center,Frozen Yogurt Shop,Diner,Comfort Food Restaurant,Construction & Landscaping


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [75]:
df_north.shape

(24, 6)

In [76]:
# set number of clusters
kclusters = 5

north_grouped_clustering = north_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:24] 

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 4, 2, 2, 1, 1, 2, 2, 2, 3, 1,
       0, 2])

In [77]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Pet Store,Shopping Mall,Diner,Ice Cream Shop,Deli / Bodega,Park,Frozen Yogurt Shop,Pharmacy,Pizza Place
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Furniture / Home Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Gift Shop,Cosmetics Shop
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Sandwich Place,Liquor Store,Comfort Food Restaurant,Café,Butcher,Pharmacy,Pizza Place
3,"CFB Toronto,Downsview East",Airport,Park,Bus Stop,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
4,Don Mills North,Caribbean Restaurant,Basketball Court,Pool,Café,Japanese Restaurant,Gym / Fitness Center,Frozen Yogurt Shop,Diner,Comfort Food Restaurant,Construction & Landscaping
5,Downsview Central,Food Truck,Business Service,Baseball Field,Korean Restaurant,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
6,Downsview Northwest,Grocery Store,Athletics & Sports,Liquor Store,Discount Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
7,Downsview West,Grocery Store,Bank,Shopping Mall,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
8,"Flemingdon Park,Don Mills South",Coffee Shop,Gym,Asian Restaurant,Beer Store,Sporting Goods Shop,Clothing Store,Chinese Restaurant,General Entertainment,Dim Sum Restaurant,Japanese Restaurant
9,Glencairn,Italian Restaurant,Bakery,Pub,Japanese Restaurant,Park,Arcade,Fried Chicken Joint,Diner,Coffee Shop,Comfort Food Restaurant


In [63]:
df_north

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,2
1,M2J,North York,"Oriole,Henry Farm,Fairview",43.778517,-79.346556,2
2,M2K,North York,Bayview Village,43.786947,-79.385975,2
3,M2L,North York,"York Mills,Silver Hills",43.757490,-79.374714,1
4,M2M,North York,"Willowdale,Newtonbrook",43.789053,-79.408493,2
5,M2N,North York,Willowdale South,43.770120,-79.408493,2
6,M2P,North York,York Mills West,43.752758,-79.400049,2
7,M2R,North York,Willowdale West,43.782736,-79.442259,2
8,M3A,North York,Parkwoods,43.753259,-79.329656,2
9,M3B,North York,Don Mills North,43.745906,-79.352188,1


In [78]:
df_north.shape

(24, 6)

In [79]:
north_merged = df_north
# add clustering labels
north_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_merged = north_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
north_merged.head(10) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,2,Dog Run,Athletics & Sports,Pool,Mediterranean Restaurant,Golf Course,Fried Chicken Joint,Food Truck,Coffee Shop,General Entertainment,Comfort Food Restaurant
1,M2J,North York,"Oriole,Henry Farm,Fairview",43.778517,-79.346556,2,Clothing Store,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Bakery,Restaurant,Bus Station,Food Court,Dessert Shop,Liquor Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,2,Chinese Restaurant,Bank,Café,Japanese Restaurant,Furniture / Home Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Gift Shop,Cosmetics Shop
3,M2L,North York,"York Mills,Silver Hills",43.757490,-79.374714,1,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,M2M,North York,"Willowdale,Newtonbrook",43.789053,-79.408493,2,Gym,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
5,M2N,North York,Willowdale South,43.770120,-79.408493,2,Ramen Restaurant,Café,Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Lounge,Plaza,Pharmacy,Pet Store
6,M2P,North York,York Mills West,43.752758,-79.400049,2,Park,Bank,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
7,M2R,North York,Willowdale West,43.782736,-79.442259,2,Pizza Place,Pharmacy,Coffee Shop,Butcher,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
8,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
9,M3B,North York,Don Mills North,43.745906,-79.352188,1,Caribbean Restaurant,Basketball Court,Pool,Café,Japanese Restaurant,Gym / Fitness Center,Frozen Yogurt Shop,Diner,Comfort Food Restaurant,Construction & Landscaping


#### let's visualize the resulting clusters


In [80]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_merged['Latitude'], north_merged['Longitude'], north_merged['Neighbourhood'], north_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Clusters


### Clusters No.0

In [81]:
north_merged.loc[north_merged['Cluster Labels'] == 0, north_merged.columns[[2] + list(range(5, north_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Humber Summit,0,Pizza Place,Empanada Restaurant,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega


### Clusters No.1

In [82]:
north_merged.loc[north_merged['Cluster Labels'] == 1, north_merged.columns[[2] + list(range(5, north_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"York Mills,Silver Hills",1,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
9,Don Mills North,1,Caribbean Restaurant,Basketball Court,Pool,Café,Japanese Restaurant,Gym / Fitness Center,Frozen Yogurt Shop,Diner,Comfort Food Restaurant,Construction & Landscaping
15,Downsview Central,1,Food Truck,Business Service,Baseball Field,Korean Restaurant,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
16,Downsview Northwest,1,Grocery Store,Athletics & Sports,Liquor Store,Discount Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
21,"Upwood Park,Maple Leaf Park,North Park",1,Park,Construction & Landscaping,Bakery,Basketball Court,Women's Store,Dog Run,Comfort Food Restaurant,Cosmetics Shop,Cupcake Shop,Deli / Bodega


### Clusters No.2

In [83]:
north_merged.loc[north_merged['Cluster Labels'] == 2, north_merged.columns[[2] + list(range(5, north_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hillcrest Village,2,Dog Run,Athletics & Sports,Pool,Mediterranean Restaurant,Golf Course,Fried Chicken Joint,Food Truck,Coffee Shop,General Entertainment,Comfort Food Restaurant
1,"Oriole,Henry Farm,Fairview",2,Clothing Store,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Bakery,Restaurant,Bus Station,Food Court,Dessert Shop,Liquor Store
2,Bayview Village,2,Chinese Restaurant,Bank,Café,Japanese Restaurant,Furniture / Home Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Gift Shop,Cosmetics Shop
4,"Willowdale,Newtonbrook",2,Gym,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
5,Willowdale South,2,Ramen Restaurant,Café,Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Lounge,Plaza,Pharmacy,Pet Store
6,York Mills West,2,Park,Bank,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
7,Willowdale West,2,Pizza Place,Pharmacy,Coffee Shop,Butcher,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
8,Parkwoods,2,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
10,"Flemingdon Park,Don Mills South",2,Coffee Shop,Gym,Asian Restaurant,Beer Store,Sporting Goods Shop,Clothing Store,Chinese Restaurant,General Entertainment,Dim Sum Restaurant,Japanese Restaurant
11,"Bathurst Manor,Downsview North,Wilson Heights",2,Coffee Shop,Pet Store,Shopping Mall,Diner,Ice Cream Shop,Deli / Bodega,Park,Frozen Yogurt Shop,Pharmacy,Pizza Place


### Clusters No.3

In [84]:
north_merged.loc[north_merged['Cluster Labels'] == 3, north_merged.columns[[2] + list(range(5, north_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Glencairn,3,Italian Restaurant,Bakery,Pub,Japanese Restaurant,Park,Arcade,Fried Chicken Joint,Diner,Coffee Shop,Comfort Food Restaurant


### Clusters No.4

In [85]:
north_merged.loc[north_merged['Cluster Labels'] == 4, north_merged.columns[[2] + list(range(5, north_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"York University,Northwood Park",4,Coffee Shop,Caribbean Restaurant,Furniture / Home Store,Metro Station,Massage Studio,Bar,Diner,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop


# Report

## Methodology

The methodology I used is pretty simple. As illustrated above, I used K-Means clusttering algorithm to cluster based on the amenities, cluster the neighbourhoods which have similar characteristics. As we know user preferences are taken into consideration such as Banks, Shopping Malls, Bus Station, Grocery, Resturants, Coffee Shops, Deli Foods/ Bakeries, Figure out which ones are best for living or setup new office/Co-working space.

## Result

Based on the amenities available in each area, the North York combinations are divided into 5 clusters. Each cluster has a unique amenity. For example **Cluster 0** is group of amenities found in any key junction of the neighborhood with Pizza Place, Restaurant, Clothing Store, Coffee Shop. **Cluster 1** is group of amenities, but more varied kind of place, Like CFB Toronto is very near to the Airport. That means if you have business usually have travel. that awesome place. **Cluster 2** is group of amenities, but should like to be resident not like the office, but it could have co-working space. **Cluster 3** is group of amenities have a lot of entertainment and relax place. such as Restaurant, Pub, Park and Acade. **Cluster 4** is group of amenities found in education institute so this place should have a Co-working Space and office.

## Conclusion

After examination of the clusters we discovered that the 1st choice is 'Bayview Village' in Cluster No.2 is closest match and suitable option for recommendation and will be shown as top priority. Because it's near a place of 2 Restaurant like Japanese and Chinese Restaurant. And the important it near Bank. Otherwise have a place to relax like Coffee shope. 2nd choice is "York University,Northwood Park" in Cluster No.4, Because this place it nearby around university. if we're start-up this place. it very useful to work with new grad student ,And it's have Coffee shop and Caribbean Restaurant to dinner or meeting.